# Création du Dataframe + initialisation de la Data

In [76]:
import pandas as pd
import json
from numpyencoder import NumpyEncoder

Df = pd.read_csv('planningTestV2.csv', sep=';')
Df.drop(columns=Df.columns[-4:], inplace=True)
Df.reset_index()
Df.fillna(0, inplace=True)


NomMajeurs = "CONCEP.LOGICIELLE/BIG DATA","ROBOTIQUE","ELECTRONIQUE ET SYST EMB","INFRA DES RESEAUX","IMAGE" 
Groupes = "Grp A","Grp B","Grp C","Grp D"

def CreateJson(NomMajeurs,Groupes):
    Data = []
    for i in NomMajeurs:
            Data.append({"NomMajeurs":i,})
    return Data

Data = CreateJson(NomMajeurs, Groupes)
print(Data[0]['NomMajeurs'])
Df

CONCEP.LOGICIELLE/BIG DATA


,LUNDI,MARDI,MERCREDI,JEUDI,VENDREDI
0,30-janv,31-janv,01-févr,02-févr,03-févr
1,8h-12h15,8h-12h15,8h-12h15,8h-12h15,8h-12h15
2,0,0,0,CONCEP.LOGICIELLE/BIG DATA,CONCEP.LOGICIELLE/BIG DATA
3,Introduction,Introduction,Introduction,Administration Système,0
4,Administration Système,Administration Système,Administration Système,0,0
...,...,...,...,...,...
67,0,Optimisation numérique et problème inverse,T.DUPONT,0,Pas de cours
68,0,0,Salle I304,0,0
69,0,14h45 - 17h45,0,0,0
70,0,E.VAN REETH /S.MAZAURIC,0,0,0


In [92]:
def createSchedule(Df,Data) :
    DicoJour = {}
    IsAprem = False
    LstSemaine = []
    for obj in Df :
        LstMatin = []
        LstAprem = []
        LstTemp = []

        for case in Df[obj] :
            if case != 0 :
                LstTemp.append(case)

        for index,value in enumerate(LstTemp) :
            if index == 0 :
                DicoJour["jour"] = obj +"-" + value
            else :
                if value == '13h10-18h' or value == '13h10-18h' :
                  IsAprem = True
                elif value == '8h-12h15' :
                    IsAprem = False
                if IsAprem :
                    LstAprem.append(value)
                else :
                    LstMatin.append(value)
        DicoJour["Matin"] = LstMatin.copy()
        DicoJour["Aprem"] = LstAprem.copy()
        LstSemaine.append(DicoJour.copy())
    print(LstSemaine)
    return LstSemaine
LstSemaine = createSchedule(Df,Data)

[{'jour': 'LUNDI-30-janv', 'Matin': ['8h-12h15', 'Introduction', 'Administration Système', 'G.MOREL', 'Grp A  Salle G102 ', 'Grp B  Salle G108 ', 'Grp C  Salle I304 ', 'Grp D  Salle G103'], 'Aprem': ['13h10-18h', 'LV', '(Voir planning)']}, {'jour': 'MARDI-31-janv', 'Matin': ['8h-12h15', 'Introduction', 'Administration Système', 'G.MOREL', 'Grp A  Salle G102 ', 'Grp B  Salle I300', 'Grp C  Salle I207 ', 'Grp D  Salle G103', '13h30-17h45', 'CONCEP.LOGICIELLE/BIG DATA', 'Bases de Données ', 'J.CHEVALIER/M.LAGARRIGUE', 'Salles  G102- G103', 'ROBOTIQUE', 'Administration système et gestion de codes', 'F.JUMEL', 'Salle G108', 'ELECTRONIQUE ET SYST EMB', 'Pas de cours', 'INFRA DES RESEAUX ', 'Architecture des réseaux locaux LAN', 'N.DANIEL ', 'Salle B121', 'R.FANTAUZZI', 'Salle I304', 'IMAGE ', 'Rentrée ', '13h30 - 14h30', 'Optimisation numérique et problème inverse', '14h45 - 17h45', 'E.VAN REETH /S.MAZAURIC', 'Salle F002'], 'Aprem': []}, {'jour': 'MERCREDI-01-févr', 'Matin': ['8h-12h15', 'In

In [101]:
def filtreMaj(LstSemaine,Data) :
    Majeur = 0
    
    for Jour in LstSemaine :   
        print(Jour["Matin"])
        for Case in Jour["Matin"] :
            if Case == "CONCEP.LOGICIELLE/BIG DATA" :
                Majeur = 1
            elif Case == "ROBOTIQUE" :
                Majeur = 2
            elif Case == "ELECTRONIQUE ET SYST EMB" :
                Majeur = 3
            elif Case == "INFRA DES RESEAUX" :
                Majeur = 4
            elif Case == "IMAGE" :
                Majeur = 5
            else :
                Majeur = 0
            Data[Majeur]["jour"].append(Jour["jour"])
        

filtreMaj(LstSemaine)

['8h-12h15', 'Introduction', 'Administration Système', 'G.MOREL', 'Grp A  Salle G102 ', 'Grp B  Salle G108 ', 'Grp C  Salle I304 ', 'Grp D  Salle G103']
['8h-12h15', 'Introduction', 'Administration Système', 'G.MOREL', 'Grp A  Salle G102 ', 'Grp B  Salle I300', 'Grp C  Salle I207 ', 'Grp D  Salle G103', '13h30-17h45', 'CONCEP.LOGICIELLE/BIG DATA', 'Bases de Données ', 'J.CHEVALIER/M.LAGARRIGUE', 'Salles  G102- G103', 'ROBOTIQUE', 'Administration système et gestion de codes', 'F.JUMEL', 'Salle G108', 'ELECTRONIQUE ET SYST EMB', 'Pas de cours', 'INFRA DES RESEAUX ', 'Architecture des réseaux locaux LAN', 'N.DANIEL ', 'Salle B121', 'R.FANTAUZZI', 'Salle I304', 'IMAGE ', 'Rentrée ', '13h30 - 14h30', 'Optimisation numérique et problème inverse', '14h45 - 17h45', 'E.VAN REETH /S.MAZAURIC', 'Salle F002']
['8h-12h15', 'Introduction', 'Administration Système', 'G.MOREL', 'Grp A  Salle G108 ', 'Grp B  Salle I300', 'Grp C  Salle I207 ', 'Grp D  Salle G103', '13h30-17h45', 'CONCEP.LOGICIELLE/BIG D

In [78]:
with open('Datatest.json', 'w+') as f:
    json.dump(Data, f, indent=4, cls=NumpyEncoder)

